In [24]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

First API - call 
input: league







In [43]:
def get_teams_by_league(league: str) -> tuple[list[str], pd.DataFrame]:
    seasons = ['1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']
    dfs = []
    for s in seasons:
        url = f"https://www.football-data.co.uk/mmz4281/{s}/{league}.csv"
        try:
            tmp = pd.read_csv(url)
        except:
            try:
                tmp = pd.read_csv(url, encoding='latin1')
            except:
                print(f"❌ Misslyckades att ladda {league} {s}")
                continue
        dfs.append(tmp)

    if not dfs:
        return [], pd.DataFrame()

    df = pd.concat(dfs, ignore_index=True)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
    df = df[df['Date'].notna()]
    df = df.sort_values(by='Date')
    teams = pd.unique(pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna())
    return sorted(teams.tolist()), df

In [39]:
def create_league_table(league: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    Bygger en ligatabell (Played, Won, Drawn, Lost, Goals For/Against, GD, Points)
    för den angivna ligan utifrån den DataFrame som skickas in.
    """
    df_work = df.copy()
    df_work['Date'] = pd.to_datetime(df_work['Date'], dayfirst=True, errors='coerce')
    now = datetime.now()

    if now.month < 8:
        start, end = datetime(now.year-1, 8, 1), datetime(now.year, 7, 1)
    else:
        start, end = datetime(now.year, 8, 1), datetime(now.year+1, 7, 1)

    matches = df_work[
        (df_work['Div'] == league) &
        (df_work['Date'] >= start) &
        (df_work['Date'] < end)
    ]

    table: dict[str, dict[str, int]] = {}
    for _, m in matches.iterrows():
        h, a, hg, ag = m['HomeTeam'], m['AwayTeam'], m['FTHG'], m['FTAG']
        for team in (h, a):
            table.setdefault(team, {
                'Played': 0, 'Won': 0, 'Drawn': 0, 'Lost': 0,
                'Goals For': 0, 'Goals Against': 0,
                'Goal Difference': 0, 'Points': 0
            })
        table[h]['Played'] += 1
        table[a]['Played'] += 1
        table[h]['Goals For']        += hg
        table[h]['Goals Against']    += ag
        table[a]['Goals For']        += ag
        table[a]['Goals Against']    += hg
        table[h]['Goal Difference']  += (hg - ag)
        table[a]['Goal Difference']  += (ag - hg)
        if hg > ag:
            table[h]['Won']    += 1; table[h]['Points'] += 3
            table[a]['Lost']   += 1
        elif hg < ag:
            table[a]['Won']    += 1; table[a]['Points'] += 3
            table[h]['Lost']   += 1
        else:
            table[h]['Drawn']  += 1; table[h]['Points'] += 1
            table[a]['Drawn']  += 1; table[a]['Points'] += 1

    league_df = pd.DataFrame.from_dict(table, orient='index')
    league_df.index.name = 'Team'
    return league_df.reset_index().sort_values(
        by=['Points', 'Goal Difference', 'Goals For'],
        ascending=False
    )

Secound API - call 
input: Hometeam, Awayteam











In [40]:
def get_fixture_by_league(league: str):
    url = 'https://www.football-data.co.uk/fixtures.csv'
    try:
        df = pd.read_csv(url, usecols=['Div', 'HomeTeam', 'AwayTeam', 'Date', 'Time'])
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
    today = pd.Timestamp.today().normalize()
    df = df[df['Date'] >= today]

    league_fixtures = df[df['Div'] == league]

    if league_fixtures.empty:
        return []

    fixtures_list = league_fixtures.to_dict(orient='records')
    return fixtures_list

In [41]:

def get_last_five_results(df: pd.DataFrame) -> list[dict]:
    required_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("❌ DataFrame saknar nödvändiga kolumner: " + ", ".join(required_columns))
    

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date'])

    # Sortera efter datum baklänges
    df_sorted = df.sort_values('Date', ascending=False)

    # Ta de 5 senaste matcherna
    last_five = df_sorted.head(5)

    # Skapa en lista med matchresultaten
    results = []
    for _, row in last_five.iterrows():
        results.append({
            "date": row['Date'].strftime('%Y-%m-%d'),
            "home_team": row['HomeTeam'],
            "away_team": row['AwayTeam'],
            "home_goals": int(row['FTHG']),
            "away_goals": int(row['FTAG']),
            "result": row['FTR'] 
        })

    return results

In [ ]:
def get_team_form(df, team, num_matches=10):
    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()
    team_matches['Date'] = pd.to_datetime(team_matches['Date'], errors='coerce')
    team_matches = team_matches.sort_values(by='Date', ascending=False).dropna(subset=['Date']).head(num_matches)

    form = []
    for _, row in team_matches.iterrows():
        is_home = row['HomeTeam'] == team
        goals_for = row['FTHG'] if is_home else row['FTAG']
        goals_against = row['FTAG'] if is_home else row['FTHG']

        if goals_for > goals_against:
            form.append('W')
        elif goals_for == goals_against:
            form.append('D')
        else:
            form.append('L')

    return form[::-1]

In [45]:
def get_home_vs_away_summary(df, home_team, away_team):

    matcher = df[(df["HomeTeam"] == home_team) & (df["AwayTeam"] == away_team)]

    resultat = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

    for _, rad in matcher.iterrows():
        mål_hemma = rad["FTHG"]
        mål_borta = rad["FTAG"]

        if mål_hemma > mål_borta:
            resultat["Vinster"] += 1
        elif mål_hemma < mål_borta:
            resultat["Förluster"] += 1
        else:
            resultat["Oavgjorda"] += 1

    return resultat

In [47]:
def create_league_table_for_current_year(league, df):
    current_year = datetime.now().year
    
    if datetime.now().month < 8:
        start_date = f"{current_year - 1}-08-01" 
        end_date = f"{current_year}-07-01"   
    else:
        start_date = f"{current_year}-08-01"      
        end_date = f"{current_year + 1}-07-01"   

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    matches_df = df[(df['Div'] == league) & 
                    (pd.to_datetime(df['Date']) >= start_date) & 
                    (pd.to_datetime(df['Date']) < end_date)]
    
    return matches_df

In [56]:
def get_avg_goals_summary(df, league, team_name):
    matches_df = create_league_table_for_current_year(league, df)

    home_matches = matches_df[matches_df['HomeTeam'] == team_name]
    home_goals_avg = home_matches['FTHG'].mean() if not home_matches.empty else 0

    away_matches = matches_df[matches_df['AwayTeam'] == team_name]
    away_goals_avg = away_matches['FTAG'].mean() if not away_matches.empty else 0

    total_goals = home_matches['FTHG'].sum() + away_matches['FTAG'].sum()
    total_matches = len(home_matches) + len(away_matches)
    overall_avg = total_goals / total_matches if total_matches > 0 else 0

    return {
        "home_avg_goals": round(home_goals_avg, 2),
        "away_avg_goals": round(away_goals_avg, 2),
        "overall_avg_goals": round(overall_avg, 2),
    }

In [57]:
def get_avg_goals_in_head_to_head(df, league, home_team, away_team):
    head_to_head = df[
        (df['Div'] == league) &
        (df['HomeTeam'] == home_team) &
        (df['AwayTeam'] == away_team)
    ]

    if len(head_to_head) > 0:
        avg_home_goals = head_to_head['FTHG'].mean() 
        avg_away_goals = head_to_head['FTAG'].mean()  
    else:
        avg_home_goals = 0
        avg_away_goals = 0

    return {
        'avg_home_goals_vs_away': avg_home_goals,
        'avg_away_goals_vs_home': avg_away_goals
    }